In [175]:
import json
from google.colab import drive
import numpy as np
import pandas as pd 
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,CuDNNLSTM,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import concatenate, SpatialDropout1D
from keras.layers.core import  Activation
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tqdm
from keras.layers import Attention
from tensorflow.keras.layers import Input
from keras.layers import Lambda
from keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [176]:
#Считываю сохраненный датафрейм
f=open('/content/data.csv')
data=pd.read_csv(f)

In [ ]:
data

In [177]:
#Добавляю в начало и конец каждого названия подборки begin и end что бы затем   
data['title'] = data['title'].apply(lambda x: 'begin ' + x \
        + ' end')


In [178]:
#Разделяю данные на тренеровочные и валидационные данные
train_text, test_text, train_y, test_y = train_test_split(data['headline'],data['title'],test_size = 0.4,random_state=42)

In [179]:
max_len_text=100
max_len_summary=20

In [180]:
#Создаю токинайзеры , убираю из тексата символы
x_tokenizer=Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True)
x_tokenizer.fit_on_texts(list(train_text))

#convert text sequences into integer sequences
train_text    =   x_tokenizer.texts_to_sequences(train_text) 
test_text   =   x_tokenizer.texts_to_sequences(test_text)

#padding zero upto maximum length
train_x    =   pad_sequences(train_text,  maxlen=max_len_text, padding='post') 
test_x   =   pad_sequences(test_text, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

In [181]:
#preparing a tokenizer for summary on training data 
y_tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True)
y_tokenizer.fit_on_texts(list(train_y))

#convert summary sequences into integer sequences
train_y    =   y_tokenizer.texts_to_sequences(train_y) 
test_y   =   y_tokenizer.texts_to_sequences(test_y) 

#padding zero upto maximum length
train_y    =   pad_sequences(train_y, maxlen=max_len_summary, padding='post')
test_y   =   pad_sequences(test_y, maxlen=max_len_summary, padding='post')

y_voc_size  =   len(y_tokenizer.word_index) +1

In [182]:
x_tr = pad_sequences(train_x,  maxlen=max_len_text, padding='post')
x_val = pad_sequences(test_x, maxlen=max_len_text, padding='post')
y_tr = pad_sequences(train_y, maxlen=max_len_summary, padding='post')
y_val = pad_sequences(test_y, maxlen=max_len_summary, padding='post')


In [11]:
type(x_tr[0])

numpy.ndarray

In [ ]:
latent_dim = 300
embedding_dim = 200


encoder_inputs = Input(shape=(max_len_text, ))

enc_emb = Embedding(x_voc_size, embedding_dim,
                    trainable=True)(encoder_inputs)

encoder_lstm1 = LSTM(latent_dim, return_sequences=True,
                     return_state=True, dropout=0.4,
                     recurrent_dropout=0.4)

(encoder_output1, state_h1, state_c1) = encoder_lstm1(enc_emb)

encoder_lstm2 = LSTM(latent_dim, return_sequences=True,
                     return_state=True, dropout=0.4,
                     recurrent_dropout=0.4)
(encoder_output2, state_h2, state_c2) = encoder_lstm2(encoder_output1)

encoder_lstm3 = LSTM(latent_dim, return_state=True,
                     return_sequences=True, dropout=0.4,
                     recurrent_dropout=0.4)
(encoder_outputs, state_h, state_c) = encoder_lstm3(encoder_output2)


decoder_inputs = Input(shape=(None, ))


dec_emb_layer = Embedding(y_voc_size, embedding_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True,
                    return_state=True, dropout=0.4,
                    recurrent_dropout=0.2)
(decoder_outputs, decoder_fwd_state, decoder_back_state) = \
    decoder_lstm(dec_emb, initial_state=[state_h, state_c])


decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
model.summary()

In [184]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')


In [ ]:
for i in range (15):

    history = model.fit(
    [x_tr, y_tr[:, :-1]],
    y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)[:, 1:],
    epochs=5,
    batch_size=20,
    validation_data=([x_val, y_val[:, :-1]],
                     y_val.reshape(y_val.shape[0], y_val.shape[1], 1)[:
                     , 1:]),
    )

In [187]:
reverse_target_word_index = y_tokenizer.index_word
reverse_source_word_index = x_tokenizer.index_word
target_word_index = y_tokenizer.word_index


In [188]:
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs,
                      state_h, state_c])

decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_hidden_state_input = Input(shape=(100, latent_dim))

dec_emb2 = dec_emb_layer(decoder_inputs)

(decoder_outputs2, state_h2, state_c2) = decoder_lstm(dec_emb2,
        initial_state=[decoder_state_input_h, decoder_state_input_c])

decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,
                      decoder_state_input_h, decoder_state_input_c],
                      [decoder_outputs2] + [state_h2, state_c2])

In [189]:
#создает тему для этих заголовков
def decode_sequence(input_seq):

    (e_out, e_h, e_c) = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))


    target_seq[0, 0] = target_word_index['begin']

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        (output_tokens, h, c) = decoder_model.predict([target_seq]
                + [e_out, e_h, e_c])

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if sampled_token != 'end':
            decoded_sentence += ' ' + sampled_token

        if sampled_token == 'end' or len(decoded_sentence.split()) \
            >= 12 - 1:
            stop_condition = True

        
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        (e_h, e_c) = (h, c)

    return decoded_sentence


In [190]:
# превращает последовательность цифр в название подборки
def seq2summary(input_seq):
    newString = ''
    for i in input_seq:
        if i != 0 and i != target_word_index['begin'] and i \
            != target_word_index['end']:
            newString = newString + reverse_target_word_index[i] + ' '

    return newString

In [191]:
# Преврашает последовательность цифр в тексты заголовков
def seq2text(input_seq):
    newString = ''
    for i in input_seq:
        if i != 0:
            newString = newString + reverse_source_word_index[i] + ' '

    return newString

In [ ]:
for i in range(0,20):
    print ('Заголовки:', seq2text(x_tr[i]))
    print ('Оригинальное название подборки:', seq2summary(y_tr[i]))
    print ('Сгенерированное название подборки:', decode_sequence(x_tr[i].reshape(1,100)))
    print ('\n')

In [233]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [234]:
model.save('/content/drive/My Drive/model1/')

INFO:tensorflow:Assets written to: /content/drive/My Drive/model1/assets


In [235]:
decoder_model.save('/content/drive/My Drive/decoder_model/')
encoder_model.save('/content/drive/My Drive/encoder_model/')

INFO:tensorflow:Assets written to: /content/drive/My Drive/decoder_model/assets
INFO:tensorflow:Assets written to: /content/drive/My Drive/encoder_model/assets


In [236]:
import pickle

# saving
with open('x_tokenizer.pickle', 'wb') as handle:
    pickle.dump(x_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [237]:
# saving
with open('y_tokenizer.pickle', 'wb') as handle:
    pickle.dump(y_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)